데이터 셋 다운로드

In [1]:
import os
import tarfile
import urllib

down_root = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
housing_path = os.path.join('datasets', 'housing')
housing_url = down_root + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url = housing_url, housing_path = housing_path) : 
    os.makedirs(housing_path, exist_ok = True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = os.path.join('data'))
    housing_tgz.close()


In [2]:
fetch_housing_data()

데이터 셋 불러오기

In [3]:
import pandas as pd

def load_data(data_path, data_name) : 
    csv_path = os.path.join(data_path, data_name)
    return pd.read_csv(csv_path)

data_path = os.path.join('data')
data_name = 'housing.csv'

housing = load_data(data_path = data_path, data_name = data_name)

In [4]:
housing['ocean_proximity'].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64